In [3]:
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import time

In [4]:
#Function to calculate the manhattan distance between 2 points
def mandist(a0, a1, b0, b1):
    return abs(a0-b0) + abs(a1-b1)

In [8]:
#Function to measure distance to nearest station 
def shortestsum(p):
    #Filling the distance matrix the longest possible distace to start
    dist = np.zeros((n,n))
    dist.fill(2*n)
    for i in range(k): #iterate for each station
        for x in range(n):
            for y in range(n): #and for each intersection
                thisdist = mandist(p[2*(i)], p[2*(i)+1], x,y) #find the distance from each point to each station
                if dist[x,y] > thisdist:
                    dist[x,y] = thisdist #replace the shortest distance for that intersection 
                    #if thisdist is shorter than previously found           
    return np.sum(dist)

In [9]:
#Function that switches the coordinates to find a more optimal solution than current
def solver(p, epoch):
    #Take the current configuration to be baseline for the minimum sum of shortest distances
    bestdist = shortestsum(p)
    bestpoints = p
    for i in range(epoch):
        for j in range(len(p)):
            #Frequency of the suboptimal move, this can be tweaked more.
            #If this value is 0, a coordinate value will increase even if it's not optimal
            #If it is 1, the value will decrease even if it's not optimal
            bump = random.randint(0, int(k*n))
            #Find the current sum of shortest distance before changing anything
            curr = shortestsum(p)
            #Try increasing the coordinate value first
            p[j] = p[j]+1
            #Calculate the new sum of shortest distances
            newsum = shortestsum(p)
            if((p[j] >= 0) & (p[j] < n) & ((newsum <= curr) or bump == 0)):
                #This means that the newly generated coordinate value decreses the sum of distance and is more optimal
                continue
            else:
                #This means that it doesn't decrease the shortest sum so let's decrease the coordinate value instead
                p[j] = p[j]-2
                newsum = shortestsum(p)
                if((p[j] >= 0) & (p[j] < n) & ((newsum <= curr) or bump == 1)):
                    #Decreasing it made the sum of shortest distances smaller
                    continue
                else:
                    #Neither increasing it or decreasing made it better, so it's better off to leave it as it was
                    p[j] = p[j]+1
            #If the newly found coordinate values provide a more optimal solution, replace the best set of points
            if newsum < bestdist:
            #    print("")
            #    print("New best solution found!! After ", i, " epochs:")
                bestpoints = list(p)
                bestdist = newsum
            #    print("bestpoints: ", bestpoints)
            #    print("bestdist: ", bestdist)
    return np.array(bestpoints), bestdist

In [ ]:
#Find an optimal arrangement for some n and k:
n = 30
k = 4
#Start a running total of the results for iterating it many times
total = 0
start = time.time()
for i in range(50):
    #Compress the problem by a scale of k (can use any value but k is a good starting point)
    oldN = n
    n = int(n/k)
    #Start the smaller problem with a completely random set of points
    p = np.random.randint(n, size=2*k)
    #Run it with a smaller n and get a solution
    p = solver(p, 10)[0]
    #Extrapolate the solution to the bigger problem and use it as a starting point
    p = np.multiply(p, k)
    n = oldN
    #Run it full scale
    bestpoints, bestdist = solver(p, 20)
    total = total + bestdist
end = time.time()
print(total/50)
print(end-start)

In [ ]:
n = 30
k = 4
total = 0
start = time.time()
#The same thing as above but the starting points are completely random
for i in range(50):
    p = np.random.randint(n, size = 2*k)
    bestpoints, bestdist = solver(p, 21)
    total = total + bestdist
end = time.time()
print(total/50)
print(end - start)

In [314]:
#Heuristic functions that ultimately failed
#They would calculate the sum of shortest distances for either the top, bottom, left or right row/column
#And subtract or add them to try to approximate the change in the total sum from moving a station by one unit
def distTopPlus(j, p):
    if j%2 == 1:
        total = 0
        for c in range(n):
             #subtract 1 from p[j] since we want the distance from the old coordinate
            total = total + mandist(p[j-1], p[j]-1, c, n)
    else:
        total = 0
        for c in range(n):
            total = total + mandist(p[j]-1, p[j+1], n, c)
    return total

def distBottomPlus(j,p):
    if j%2 == 1:
        total = 0
        for c in range(n):
            total = total + mandist(p[j-1], p[j], c, 0)
    else:
        total = 0
        for c in range(n):
            total = total + mandist(p[j], p[j+1], 0, c)
    return total

def distTopMinus(j, p):
    if j%2 == 1:
        total = 0
        for c in range(n):
             #subtract 1 from p[j] since we want the distance from the old coordinate
            total = total + mandist(p[j-1], p[j]-1, c, 0)
    else:
        total = 0
        for c in range(n):
            total = total + mandist(p[j]-1, p[j+1], 0, c)
    return total

def distBottomMinus(j,p):
    if j%2 == 1:
        total = 0
        for c in range(n):
            total = total + mandist(p[j-1], p[j], c, 1)
    else:
        total = 0
        for c in range(n):
            total = total + mandist(p[j], p[j+1], 1, c)
    return total

In [315]:
#The function to find the solution using a heuristic distance calculation that didn't work
def solverH(p, epoch):
    bestdist = shortestsum(p)
    bestpoints = p
    bestdistH = shortestsum(p)
    for i in range(epoch):
        if i%int(epoch/(n*k)):
            bestdistH = shortestsum(p)
        for j in range(len(p)):
            bump = random.randint(0, int(n*k))
            #Generate a random number to either increase or decrease each coordinate value
            a = random.randint(0, 1)
            
            #Find the current sum of shortest distance before changing anything
            #curr = shortestsum(p)
            #Increase the specific coordinate value if a=1, decrease if 0
            if a == 1:
                p[j] = p[j]+1
                bestdistHnew = bestdistH - distTopPlus(j,p) + distBottomPlus(j,p)
                if((p[j] >= 0) & (p[j] < n) & ((bestdistHnew < bestdistH) or bump == 0)):
                    #This means that the newly generate coordinate value decreses the sum of distance and is more optimal
                    bestdistH = bestdistH - distTopPlus(j,p) + distBottomPlus(j,p)
                    print("Actual best distance: ", shortestsum(p), "Heuristic: ", bestdistH)
                else:
                    #This means that it doesn't so it's better off to leave it as we started
                    p[j] = p[j]-1 
            #Same process but if a = 0    
            else:
                p[j] = p[j]-1
                bestdistHnew = bestdistH - distBottomMinus(j,p) + distTopMinus(j,p)
                if((p[j] >= 0) & (p[j] < n) & ((bestdistHnew < bestdistH) or bump == 0)):
                    bestdistH = bestdistH - distBottomMinus(j,p) + distTopMinus(j,p)
                    print("Actual best distance: ", shortestsum(p), "Heuristic: ", bestdistH)
                else:
                    p[j] = p[j]+1
            if shortestsum(p) < bestdist:
                print("")
                print("New best solution found!! After ", i, " epochs:")
                bestpoints = list(p)
                bestdist = shortestsum(p)
                print("bestpoints: ", bestpoints)
                print("bestdist: ", bestdist)
    return np.array(bestpoints), bestdist